# Co-ocurrence of Extreme Drought and Marine Heat Waves in the California Current

# <font color=blue>Hypothesis:</font>
## <font color=blue>Will extreme droughts and marine heat waves occur simultaneously more often because of anthropogenic climate change? </font>

### We will test this hypothesis by comparinig the number of co-ocurring events in CMIP6 climate models for control, historical and future runs.

### We define extreme droughts as  years below the 10% long-term average of California 2m soil moisture July-June mean values

### We define marine heatwaves as years above the 90% long-term average of California Current Sea Surface Temperature July-June mean values

In [8]:
# Some definitions

# directory of data
#fdir = '~/Google Drive/professional/research/FARALLON_INSTITUTE_PROJECTS/2020 NOAA MAPP/Climate_extremes_sharedfigsandcode/data/FOR SCATTER PLOT/'
fdir = '/Volumes/GoogleDrive/My Drive/Climate_extremes_sharedfigsandcode/data/FOR SCATTER PLOT/'
modsst = xr.open_dataset(fdir+'original ts/picontrol_sst.nc')

In [9]:
# Libraries 

import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore') #filter some warning messages
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# personal functions


## Calculate thresholds for PI Control runs

In [10]:
def get_thre(y1,y2,prc1,prc2):
    # open data: original (with trend)

    modsst = xr.open_dataset(fdir+'original ts/picontrol_sst.nc')
    modsst.close()
    modsm2 = xr.open_dataset(fdir+'original ts/picontrol_sm.nc')
    modsm2.close()

    # select period 
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1,y2))
    mhw_thr = list()
    drg_thr = list()
    models = list()

    # Calculate thresholds for each model
    for ix,i in enumerate(modsm2.model.values):
        tmp1= modsst.sel(model=i).sst.values
        tmp2= modsm2.sel(model=i).sm.values
        
        mhw_thr.append(np.percentile(tmp1,prc1))
        drg_thr.append(np.percentile(tmp2,prc2))

        models.append(i)
    
    return mhw_thr, drg_thr, models

In [11]:
# calculate number of events for control, historical and future
# based on pi control thresholds

def freq_coevents(fdir, runs, mhw_thr, drg_thr, models, y1, y2):
    #pi control
    fi1 = fdir+'original ts/'+runs+'_sst.nc'
    fi2 = fdir+'original ts/'+runs+'_sm.nc'
    modsst = xr.open_dataset(fi1)
    modsst.close()
    modsm2 = xr.open_dataset(fi2)
    modsm2.close()

    # select period
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1,y2))

    nev = list()

    for ix,i in enumerate(models):
        #print(ix,i)
        tmp1= modsst.sel(model=i).sst.values
        tmp2= modsm2.sel(model=i).sm.values

        a1 = tmp1>=mhw_thr[ix]
        a2 = tmp2<=drg_thr[ix]
        tmp = np.full((len(modsst.year.values),1),1)
        tmp = tmp[a1*a2]

        nev.append(len(tmp))
    
    freq = np.array(nev)/(y2-y1+1)
    
    return freq, np.round(np.nanmean(freq),4), np.round(np.nanstd(freq),4) #mean frequency
                 #np.round(np.nanmean(nev),4), np.round(np.nanstd(nev),4) #mean no. of events

In [12]:
def freq_difruns(ny, prc1, prc2):
    
    # calculate threshold in pi control
    y2=2014
    y1=y2-ny+1
    mhw_thr, drg_thr, models = get_thre(y1,y2,prc1,prc2)

    # pi control
    print('\npiControl')
    y2=2014
    y1=y2-ny+1
    freq, mfrq, sfrq = freq_coevents(fdir,'picontrol',mhw_thr,drg_thr, models,y1,y2)
    print(mfrq,sfrq)

    # historical 1
    y1 = 1900
    y2 = y1+ny-1
    print('\nhistorical: '+str(y1)+'-'+str(y2))
    freq, mfrq, sfrq = freq_coevents(fdir,'historical',mhw_thr,drg_thr, models,y1,y2)
    print(mfrq,sfrq)

    # historical 2
    y2=2014
    y1=y2-ny+1
    print('\nhistorical 2: '+str(y1)+'-'+str(y2))
    freq, mfrq, sfrq = freq_coevents(fdir,'historical',mhw_thr,drg_thr, models,y1,y2)
    print(mfrq,sfrq)

    # future
    y2=2099
    y1=y2-ny+1
    print('\nfuture: '+str(y1)+'-'+str(y2))
    freq, mfrq, sfrq = freq_coevents(fdir,'ssp585',mhw_thr,drg_thr,  models, y1,y2)
    print(mfrq,sfrq)


# Frequency of events - original data, using pi control threshols
## percentiles 80-20

In [13]:
ny = 30
prc1 = 80
prc2 = 20

print ('\nPercentiles: '+str(prc1)+'/'+str(prc2))
print('\nProbable # events in '+str(ny)+' years'+': '+str(ny*0.04))

freq_difruns(ny, prc1, prc2)


Percentiles: 80/20

Probable # events in 30 years: 1.2

piControl
0.0197 0.0217

historical: 1900-1929
0.0788 0.1187

historical 2: 1985-2014
0.2545 0.2922

future: 2070-2099
0.5121 0.368


# Frequency of events - original data, using pi control threshols
## percentiles 90-10

In [14]:
ny = 30
prc1 = 90
prc2 = 10

print ('\nPercentiles: '+str(prc1)+'/'+str(prc2))
print('\nProbable # events in '+str(ny)+' years'+': '+str(ny*0.01))

freq_difruns(ny, prc1, prc2)


Percentiles: 90/10

Probable # events in 30 years: 0.3

piControl
0.0045 0.0114

historical: 1900-1929
0.047 0.0874

historical 2: 1985-2014
0.1773 0.2446

future: 2070-2099
0.4288 0.3881
